In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from time import time
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
%%time
# create a variable for the 20newsgroups dataset training set
newsgroups_train = fetch_20newsgroups(subset='train')

CPU times: user 258 ms, sys: 83.6 ms, total: 342 ms
Wall time: 383 ms


# pyLDAvis Analysis

In [3]:
# new variable for LDA analysis, removing meta information and pesky footers
news = fetch_20newsgroups(subset='train', shuffle=True, random_state=28,
                remove=('headers', 'footers', 'quotes'))
news_raw = news.data
print(len(news_raw))

11314


In [8]:
%%time
## create a limited set of categories of the 20
categories = ['misc.forsale', 'talk.politics.mideast', 
              'sci.space', 'rec.sport.baseball']

# create a new variable for the reduced dataset
# to 4 categories for faster analysis testing
news_reduced = fetch_20newsgroups(subset='train', shuffle=True, random_state=28,
                remove=('headers', 'footers', 'quotes'), categories=categories)

CPU times: user 1.64 s, sys: 83.3 ms, total: 1.73 s
Wall time: 1.91 s


## Vectorizing the documents

Count Vectorizer:

In [9]:
# sklearn turns a collection of text documents to a matrix of token counts.
# This allows sentences or small paragraphs to become rows.
# TF vecotrizer
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                preprocessor=None,
                                binary=False,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10,
                                encoding='utf-8',)
dtm_tf = tf_vectorizer.fit_transform(news_raw)
print(dtm_tf.shape)

# documentation: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

(11314, 9144)


documentation for Count vectorizer: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

TF-IDF Vectorizer:

In [10]:
# tf-idf vectorizer
# Converts a collection of raw documents to a matrix of TF-IDF features.
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(news_raw)
print(dtm_tfidf.shape)

/Users/jakegrosek/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1577: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)


(11314, 9144)


documentation for tf-idf vectorizer: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

## TF-IDF Document Visualization 

In [23]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tf.fit(dtm_tf)

/Users/jakegrosek/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=20, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [19]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
14     11.164821        1       1 -0.180955 -0.109945
6      10.139114        1       2 -0.061316 -0.124343
0       9.136566        1       3  0.013934  0.210275
18      7.076258        1       4 -0.103627  0.075069
13      6.911650        1       5 -0.129316 -0.081452
19      5.928623        1       6 -0.070796  0.160446
8       5.549917        1       7 -0.006708  0.214477
12      5.131104        1       8 -0.045235 -0.113258
17      4.329691        1       9 -0.094944 -0.129074
9       4.322022        1      10  0.125719  0.020179
15      4.127279        1      11 -0.092024  0.060330
5       4.032042        1      12 -0.137352 -0.112072
1       4.031012        1      13  0.013149  0.079522
7       3.894340        1      14  0.020685  0.020959
11      3.319029        1      15  0.091187  0.142276
3       3.145560        1      16  0.157685  0.083828
2       2.824126        1      17 -0.105587 -0.026256
10      2.276148        1      18  0.034392 -0.122814
4       1.698039        1      19  0.197919 -0.092564
16      0.962660        1      20  0.373188 -0.155583, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
5016  Default  4591.000000            max  4591.000000  30.0000  30.0000
2653  Default  2429.000000            edu  2429.000000  29.0000  29.0000
3523  Default  1952.000000            god  1952.000000  28.0000  28.0000
4497  Default  1206.000000            key  1206.000000  27.0000  27.0000
7677  Default  1249.000000          space  1249.000000  26.0000  26.0000
1506  Default  1430.000000            com  1430.000000  25.0000  25.0000
3153  Default  1716.000000           file  1716.000000  24.0000  24.0000
2560  Default  1109.000000          drive  1109.000000  23.0000  23.0000
3639  Default   715.000000            gun   715.000000  22.0000  22.0000
804   Default   457.000000            bhj   457.000000  21.0000  21.0000
4396  Default   875.000000          jesus   875.000000  20.0000  20.0000
3405  Default   816.000000           game   816.000000  19.0000  19.0000
457   Default   602.000000       armenian   602.000000  18.0000  18.0000
3497  Default   434.000000            giz   434.000000  17.0000  17.0000
3551  Default  1219.000000     government  1219.000000  16.0000  16.0000
8982  Default  1221.000000        windows  1221.000000  15.0000  15.0000
5924  Default  4079.000000         people  4079.000000  14.0000  14.0000
1161  Default   786.000000           card   786.000000  13.0000  13.0000
8278  Default  3005.000000          think  3005.000000  12.0000  12.0000
8532  Default   478.000000        turkish   478.000000  11.0000  11.0000
7250  Default   637.000000           scsi   637.000000  10.0000  10.0000
3986  Default   690.000000          image   690.000000   9.0000   9.0000
1322  Default   697.000000           chip   697.000000   8.0000   8.0000
8167  Default   760.000000           team   760.000000   7.0000   7.0000
1157  Default   783.000000            car   783.000000   6.0000   6.0000
4401  Default   551.000000           jews   551.000000   5.0000   5.0000
458   Default   502.000000      armenians   502.000000   4.0000   4.0000
2517  Default   705.000000            dos   705.000000   3.0000   3.0000
4620  Default  1051.000000            law  1051.000000   2.0000   2.0000
5452  Default  2582.000000            new  2582.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
5545  Topic20    78.146052            nuy    79.086678   4.6313  -4.6135
3481  Topic20    78.146052            ghj    79.086678   4.6313  -4.6135
805   Topic20    61.125131           bhjn    62.065757   4.6280  -4.8592
5325  Topic20    44.104210            mtm    45.044836   4.6221  -5.1855
5529  Topic20    38.096826           nriz    39.037452   4.6188  -5.

## TF-IDF Document Visualization 

In [24]:
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

/Users/jakegrosek/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=20, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [20]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
6      43.540430        1       1  0.194033 -0.089658
0      16.332662        1       2  0.327342  0.094995
5      12.299538        1       3  0.126914 -0.179710
15      3.751893        1       4  0.088920  0.146987
10      2.442898        1       5 -0.006074  0.075008
12      2.345169        1       6 -0.039268 -0.075630
14      2.262753        1       7 -0.044233 -0.111076
9       2.151293        1       8 -0.114646  0.053899
7       1.965352        1       9 -0.068014  0.057396
4       1.742128        1      10 -0.047578  0.025510
18      1.244926        1      11 -0.040547  0.012879
17      1.237738        1      12 -0.050310 -0.010800
13      1.206207        1      13 -0.033021  0.007351
1       1.130089        1      14 -0.054647  0.010332
11      1.129160        1      15 -0.042624 -0.009673
8       1.121394        1      16 -0.046793  0.000169
19      1.091780        1      17 -0.042886  0.001353
16      1.005387        1      18 -0.035102  0.001121
2       1.004232        1      19 -0.038579 -0.007595
3       0.994971        1      20 -0.032888 -0.002858, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
4497  Default   52.000000           key   52.000000  30.0000  30.0000
2653  Default   90.000000           edu   90.000000  29.0000  29.0000
9111  Default   66.000000          year   66.000000  28.0000  28.0000
3405  Default   51.000000          game   51.000000  27.0000  27.0000
1322  Default   42.000000          chip   42.000000  26.0000  26.0000
8167  Default   42.000000          team   42.000000  25.0000  25.0000
3523  Default   80.000000           god   80.000000  24.0000  24.0000
4355  Default   33.000000        israel   33.000000  23.0000  23.0000
8982  Default   78.000000       windows   78.000000  22.0000  22.0000
3406  Default   40.000000         games   40.000000  21.0000  21.0000
1506  Default   63.000000           com   63.000000  20.0000  20.0000
120   Default   40.000000       address   40.000000  19.0000  19.0000
7657  Default   28.000000          soon   28.000000  18.0000  18.0000
2560  Default   70.000000         drive   70.000000  17.0000  17.0000
8252  Default  112.000000        thanks  112.000000  16.0000  16.0000
1425  Default   25.000000       clipper   25.000000  15.0000  15.0000
2747  Default   26.000000    encryption   26.000000  14.0000  14.0000
7141  Default   64.000000          said   64.000000  13.0000  13.0000
1161  Default   59.000000          card   59.000000  12.0000  12.0000
4356  Default   21.000000       israeli   21.000000  11.0000  11.0000
4500  Default   24.000000          keys   24.000000  10.0000  10.0000
5016  Default   18.000000           max   18.000000   9.0000   9.0000
6080  Default   26.000000       players   26.000000   8.0000   8.0000
2704  Default   50.000000         email   50.000000   7.0000   7.0000
4695  Default   56.000000           let   56.000000   6.0000   6.0000
3837  Default   23.000000        hockey   23.000000   5.0000   5.0000
1157  Default   57.000000           car   57.000000   4.0000   4.0000
8979  Default   30.000000           win   30.000000   3.0000   3.0000
8675  Default  113.000000           use  113.000000   2.0000   2.0000
4895  Default   69.000000          mail   69.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
4205  Topic20    2.290329   instruments    5.202026   3.7899  -5.5225
6996  Topic20    2.304869        riders    5.369080   3.7646  -5.5162
9068  Topic20    1.203693       wrapped    2.919261   3.7243  -6.1658
4377  Topic20    0.853253         jamie    2.091975   3.7134  -6.5099
6378  Topic20    0.918020    projection    2.344315   3.6727  -6.4368
4707  Topic20    0.676274        liable    1.784059   3.6402  -6.7424
4432  Topic20    0.997844    journalist

## Traditional NLP??

In [25]:
## import 20newsgorups as a list?

In [4]:
%%time
from pprint import pprint
# view the dictionary keys
pprint(list(newsgroups_train))

['data', 'filenames', 'target_names', 'target', 'DESCR']
CPU times: user 576 µs, sys: 1.19 ms, total: 1.77 ms
Wall time: 990 µs


In [5]:
%%time
# understanding the length and shape of the file
print(newsgroups_train.filenames.shape)
print(newsgroups_train.target.shape)
print(newsgroups_train.target[:10])

(11314,)
(11314,)
[ 7  4  4  1 14 16 13  3  2  4]
CPU times: user 608 µs, sys: 878 µs, total: 1.49 ms
Wall time: 683 µs


In [21]:
# not necessary due to tfidf vectorization?

#news_reduced = text_cleaner(news_reduced)

In [13]:
%%time
import spacy
nlp = spacy.load('en')
nlp.max_length = 23065807
news_doc = nlp(" ".join(news_reduced["data"]))

CPU times: user 1min 49s, sys: 1min 2s, total: 2min 52s
Wall time: 4min 8s


In [15]:
%%time
news_sents = [[sent, "data"] for sent in news_doc.sents]

# Create one data frame.
sentences = pd.DataFrame(news_sents)
sentences.head()

CPU times: user 263 ms, sys: 428 ms, total: 691 ms
Wall time: 962 ms


In [6]:
%%time
#from collections import Counter
from sklearn.datasets import fetch_20newsgroups
import re

# define text cleaner function for the dataset
def text_cleaner(line:str):
    '''define the function for a text cleaner to run the news through'''
        # replaces new line with a space
    line = re.sub('\n', ' ', line)
        # removes characters outside of a-z w/ a space
    line = re.sub('[^a-z ]', ' ', line)
        # removes things within a [] bracket and replaces w/ a space
    line = re.sub("[[].*?[]]", ' ', line)
        # removes -- and replaces w/ a space
    line = re.sub(r'--', ' ', line)
        # splits sentences and makes it a list 
    line = ' '.join(line.split())
    return line

# deprecation warning had me remove \ in line 14, "[\[].*?[\]]"
# source: https://github.com/pallets/jinja/issues/646

CPU times: user 14 µs, sys: 1e+03 ns, total: 15 µs
Wall time: 19.1 µs


In [7]:
# saving for pythonic example of a good for loop
#tokens = [token for token in tokens if len(token) > 0]

In [16]:
# all the text is being labeled "data." We need it to say the proper publication
sentences.head()

,0,1
0,"(Yes, ,, the, Phobos, mission, did, return, so...",data
1,"(The, best, I, 've, seen, had, a, surface, res...",data
2,"(By, \n, the, way, ,, the, new, book, entitled...",data
3,"(The, chapter, is, co, -, authored, by, V.I., ...",data
4,"(Do, n't, \n, know, of, any, ftp, sites, with,...",data


In [25]:
%%time
# Mike--> code from our session last week
from typing import List
def bag_of_words_spacy(doc:spacy.tokens.doc.Doc)->List:
    """takes spacy doc, returns list of most common 2000 lemmas"""
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in doc
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]

# Mike--> code from our session last week
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features_df(sentences:List, common_words)->pd.DataFrame:
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['label'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(sentences[0]):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [span.lemma_
                 for span in sentence
                 if (
                     not span.is_punct
                     and not span.is_stop
                     and span.lemma_ in common_words
                 )
                ]
        

         #Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

CPU times: user 8 µs, sys: 1 µs, total: 9 µs
Wall time: 14.3 µs
